### Import/Load Data

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.contrib.learn.python.learn.datasets import base


# Load dataset
training_data = "iris_training.csv"
testing_data = "iris_test.csv"
training_set = base.load_csv_with_header(filename=training_data,
                                         features_dtype=np.float32, 
                                         target_dtype=np.int)
test_set = base.load_csv_with_header(filename=testing_data,
                                     features_dtype=np.float32,
                                     target_dtype=np.int)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
feature_name = "iris_features"
feature_columns = [tf.feature_column.numeric_column(feature_name, shape=[4])]

### Input Functions

In [3]:
def input_fn(data):
    features = {feature_name: tf.constant(data.data)}
    label = tf.constant(data.target)
    return features, label


In [4]:
train_input = lambda: input_fn(training_set)
eval_input = lambda: input_fn(test_set)

### Training w/ Linear Classifier

In [5]:
classifier = tf.estimator.LinearClassifier(
    feature_columns=feature_columns,
    n_classes=3,
    model_dir="tmp/iris")


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'tmp/iris', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x11b1116a0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [6]:
# define training, eval spec for train and evaluate including
train_spec = tf.estimator.TrainSpec(train_input, 
                                    max_steps=3000
                                    )
eval_spec = tf.estimator.EvalSpec(eval_input,
                                name='mnist-eval'
                                )                                  
# run training and evaluation
tf.estimator.train_and_evaluate(
    classifier, train_spec, eval_spec)


INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after 600 secs (eval_spec.throttle_secs) or training is finished.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into tmp/iris/model.ckpt.
INFO:tensorflow:loss = 131.83345, step = 1
INFO:tensorflow:global_step/sec: 1506.29
INFO:tensorflow:loss = 37.13907, step = 101 (0.067 sec)
INFO:tensorflow:global_step/sec: 2408.37
INFO:tensorflow:loss = 27.859367, step = 201 (0.042 sec)
INFO:tensorflow:global_step/sec: 2503.51
INFO:tensorflow:loss = 23.044888, step = 301 (0.040 sec)
INFO:tensorflow:global_step/sec: 2267.62
INFO:tensorflow:loss = 20.058025, step = 401 (0.044 sec)
INFO:tensorflow:global_step/sec: 2451.88
INFO:ten

### Training w/ Deep Neural Network Estimator

In [7]:
nn_classifier = tf.estimator.DNNClassifier(
    feature_columns=feature_columns,
    hidden_units=[64, 12, 4],
    activation_fn=tf.nn.relu,
    dropout=0.4,
    n_classes=3,
    optimizer="Adam",    
    model_dir="tmp/irisnn")


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'tmp/irisnn', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x11b138748>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [8]:
# define training, eval spec for train and evaluate including
train_spec = tf.estimator.TrainSpec(train_input, 
                                    max_steps=4200
                                    )
eval_spec = tf.estimator.EvalSpec(eval_input,
                                name='mnist-eval'
                                )                                  
# run training and evaluation
tf.estimator.train_and_evaluate(
    nn_classifier, train_spec, eval_spec)

INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after 600 secs (eval_spec.throttle_secs) or training is finished.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into tmp/irisnn/model.ckpt.
INFO:tensorflow:loss = 207.06589, step = 1
INFO:tensorflow:global_step/sec: 657.427
INFO:tensorflow:loss = 131.5281, step = 101 (0.153 sec)
INFO:tensorflow:global_step/sec: 1115.93
INFO:tensorflow:loss = 131.52809, step = 201 (0.090 sec)
INFO:tensorflow:global_step/sec: 1126.57
INFO:tensorflow:loss = 131.52808, step = 301 (0.088 sec)
INFO:tensorflow:global_step/sec: 1268.03
INFO:tensorflow:loss = 131.52808, step = 401 (0.080 sec)
INFO:tensorflow:global_step/sec: 1067.02
INFO:t

### Serving function and exporter

In [9]:
feature_spec = {feature_name: 
                tf.FixedLenFeature(shape=[4], dtype=np.float32)}
serving_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(feature_spec)

exporter = tf.estimator.LatestExporter('exporter',serving_fn) 

eval_spec = tf.estimator.EvalSpec(eval_input,
                                name='mnist-eval',
                                exporters=[exporter]
                                )     

### Re-run and export model

In [10]:
# run training and evaluation
tf.estimator.train_and_evaluate(
    nn_classifier, train_spec, eval_spec)

INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after 600 secs (eval_spec.throttle_secs) or training is finished.
INFO:tensorflow:Skipping training since max_steps has already saved.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-03-24-18:26:27
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from tmp/irisnn/model.ckpt-4200
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [10/100]
INFO:tensorflow:Evaluation [20/100]
INFO:tensorflow:Evaluation [30/100]
INFO:tensorflow:Evaluation [40/100]
INFO:tensorflow:Evaluation [50/100]
INFO:tensorflow:Evaluation [60/100]
INFO:tensorflow:Evaluation [70/100]
INFO:tensorflow:Evaluation [80/100]
INFO:tensorflow:Evaluation [90/100]
INFO:tensorflow:Evaluation [100/100]
INFO:tensorflow:Finished evaluation 

### Export Model for Prediction

In [11]:
new_samples = np.array(
      [[6.4, 3.2, 4.5, 1.5],
       [5.8, 3.1, 5.0, 1.7]], dtype=np.float32)
    
predict_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={feature_name: new_samples},
      num_epochs=1,
      shuffle=False)

predictions = list(nn_classifier.predict(input_fn=predict_input_fn))
predicted_classes = [int(p['classes']) for p in predictions]

print("New Samples, Class Predictions: {}\n".format(predicted_classes))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from tmp/irisnn/model.ckpt-4200
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
New Samples, Class Predictions: [0, 0]

